# Jupyter pglite anywidget demo

Demo of using `jupyterlite_anywidget_pglite` widget and magics.

Works in:

- Jupyter UIs (JupyterLab, Jupyter notebook)
- VS Code (via Jupyter notebook support)

## Headless Demo

In the headless demo, no HTML UI elements are added to the DOM, but the `anywidget` machinery *is* added via a call to `IPython.display.display`.

In [2]:
%load_ext jupyter_anywidget_pglite
from jupyter_anywidget_pglite import pglite_headless

In [3]:
pg_headless = pglite_headless()

postgresWidget(headless=True)

In [4]:
%%pglite_magic pg_headless
CREATE TABLE IF NOT EXISTS test  (
        id serial primary key,
        title varchar not null
      );

In [5]:
%%pglite_magic pg_headless
INSERT INTO test (title) VALUES ('dummy_headless');


In [6]:
%%pglite_magic pg_headless
SELECT * FROM test;

In [7]:
pg_headless.response

{'rows': [{'id': 1, 'title': 'dummy_headless'}],
 'fields': [{'name': 'id', 'dataTypeID': 23},
  {'name': 'title', 'dataTypeID': 1043}],
 'affectedRows': 0}

Use the `.df()` method on the widget to display a query result as a `pandas` dafaframe, if pandas is installed.

In [8]:
pg_headless.df()

,title
id,
1,dummy_headless


## Database persistence

We can persist the database using a browser's `IndexedDB` datastore, passing the name of the indexed database via an `idb=` parameter (Jupyter browser-based UIs; does not work in VSCode).

This parameter can also be used in the inline and panel display database creation steps.

In [ ]:
pg_headless_persist = pglite_headless(idb="pglitetest1")

## Inline HTML Display (Initialising cell)

We can display the widget inline as a cell output, althoug the utility of this is perhaps a little bit limited?

In [9]:
from jupyter_anywidget_pglite import pglite_inline

pg_inline = pglite_inline()

postgresWidget()

In [10]:
%%pglite_magic pg_inline
CREATE TABLE IF NOT EXISTS test  (
        id serial primary key,
        title varchar not null
      );

In [11]:
%%pglite_magic pg_inline
INSERT INTO test (title) VALUES ('dummy_inline');

In [12]:
%%pglite_magic pg_inline
SELECT * FROM test;

In [13]:
pg_inline.response

{'rows': [{'id': 1, 'title': 'dummy'}],
 'fields': [{'name': 'id', 'dataTypeID': 23},
  {'name': 'title', 'dataTypeID': 1043}],
 'affectedRows': 0}

## HTML in own panel

In a Juoyter Lab environment, we can use [`jupyterlab-sidecar`](https://github.com/jupyter-widgets/jupyterlab-sidecar) to display the widget HTML UI in its own panel (note: this does not currently work at all in VS Code).

In [14]:
from jupyter_anywidget_pglite import pglite_panel

pg_panel = pglite_panel()

In [15]:
%%pglite_magic pg_panel
CREATE TABLE IF NOT EXISTS test  (
        id serial primary key,
        title varchar not null

In [16]:
%%pglite_magic pg_panel
INSERT INTO test (title) VALUES ('dummy_panel');

In [17]:
%%pglite_magic pg_panel
SELECT * FROM test;

In [18]:
pg_panel.response

{}